In [37]:
from transformers import BertTokenizer, BertForPreTraining
import torch, torchvision
import tensorflow as tf
import pandas as pd
from datasets import Dataset
import multiprocessing


In [6]:
# Set up GPU backend
# https://pytorch.org/docs/main/notes/mps.html
# https://stackoverflow.com/questions/63423463/using-pytorch-cuda-on-macbook-pro
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
    # output expected:
    # tensor([1.], device='mps:0')

else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [42]:
# Read in events and store as dataset
df = pd.read_csv('eid_eventText.csv', usecols=['eid','event_text']).dropna(how='any',axis=0)
print(len(df))
df.drop_duplicates(subset=['event_text'],keep='first',inplace=True)
print(len(df))
dataset = Dataset.from_pandas(df).shuffle(seed=242)

516259
448453


In [41]:
# Tokenization
model = BertForPreTraining.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(reports):
    return tokenizer(reports["event_text"], return_tensors="np", truncation=True, padding="max_length")

tokenized_ds = dataset.map(tokenize_function, batched=True, num_proc=multiprocessing.cpu_count())

Map (num_proc=24): 100%|██████████| 448453/448453 [02:15<00:00, 3313.92 examples/s] 


In [ ]:
# Setup model
# https://github.com/google-research/bert#pre-training-with-bert
# https://huggingface.co/learn/nlp-course/chapter3/3
# https://huggingface.co/blog/pretraining-bert
# https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#overview
# https://www.kaggle.com/code/thierryneusius/pretraining-bert-with-hugging-face-transformers
tokenized_ds.train_test_split(test_size=0.1)

In [35]:
multiprocessing.cpu_count()

24